In [1]:
import ee, geetools

ee.Initialize()

In [2]:
vatican = ee.Geometry.Point([12.4534, 41.9033]).buffer(1)
fc = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_CODE', 110))
fc

In [4]:
vatican = ee.Geometry.Point([12.4534, 41.9033]).buffer(1)
image = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(vatican).first()
image = image.select(["B4", "B2"]).geetools.interpolateBands([0, 3000], [0, 30])
values = image.reduceRegion(ee.Reducer.mean(), vatican, 1)
print(values.getInfo())

AttributeError: 'Image' object has no attribute 'select'

In [ ]:
image.getInfo()

In [ ]:
values = image.reduceRegion(ee.Reducer.first(), vatican, 1)

In [ ]:
values.getInfo()